In [4]:
from typing import Dict, Generator, Iterator, Tuple
import numpy as np

import warnings
from keras.utils import to_categorical

warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)
from dataset import load_tokenized_sentences

STEP = 2
MAX_INPUT_LEN = 100
BATCH_SIZE = 2048

dataset1 = load_tokenized_sentences("../datasets/pickled/chars/bajki-extend_str.pickle")
dataset2 = load_tokenized_sentences("../datasets/pickled/chars/books-raw_str.pickle")
dataset = dataset2
CHARS = sorted(list(set(dataset)))
C2I = dict((c, i) for i, c in enumerate(CHARS))
I2C = dict((i, c) for i, c in enumerate(CHARS))

print("Corpus length:", len(dataset))
print("Total chars:", len(CHARS))

def text_generator(
    dataset: str,
    c2i: Dict[str, int] = C2I,
    max_len: int = MAX_INPUT_LEN,
    step: int = STEP,
    shuffle=True,
    batch_size: int = 1024,
    sparse: bool = False
) -> Iterator[Tuple[np.ndarray, np.ndarray]]:
    sequences = []
    last_chars = []
    for i in range(0, len(dataset) - max_len, step):
        sequences.append([c2i[c] for c in dataset[i : i + max_len]])
        last_chars.append(c2i[dataset[i + max_len]])
    print("Number of sequences:", len(sequences))

    chars = len(c2i)
    x = np.array(sequences, dtype=int)
    y = np.array(last_chars, dtype=int)
    while True:
        if shuffle:
            p = np.random.permutation(len(sequences))
        for i in range(0, len(sequences) - batch_size + 1, batch_size):
            indx = np.s_[i:i+batch_size]
            if shuffle:
                indx = p[indx]
            _x = to_categorical(x[indx], chars)
            _y =  y[indx]
            if not sparse: 
                _y = to_categorical(_y, chars)
            yield _x, _y

Corpus length: 46877858
Total chars: 44


In [5]:
gen = text_generator(
    dataset, C2I, MAX_INPUT_LEN, STEP, shuffle=True, batch_size=BATCH_SIZE, sparse=True
)
a, b = next(gen)
print(a.shape, b.shape)

Number of sequences: 23438879
(2048, 100, 44) (2048,)


In [10]:
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Input, Embedding, LeakyReLU
from keras.models import Sequential
from keras import optimizers, activations, regularizers, losses

def create_model(max_input_len: int, total_chars: int):
    model = Sequential([
        Input((max_input_len, len(CHARS))),
        Bidirectional(LSTM(128, return_sequences=True)),
        LSTM(256),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-2)),
        Dense(total_chars)
    ])
    model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=optimizers.Adam(1e-3))
    return model

model = create_model(MAX_INPUT_LEN, len(CHARS))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 100, 256)         177152    
 nal)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 44)                11308     
                                                                 
Total params: 779,564
Trainable params: 779,564
Non-trainable params: 0
________________________________________________

In [33]:
import tensorflow as tf

class PredictCallback(tf.keras.callbacks.Callback):
    def __init__(
        self,
        seed_text: str,
        next_chars: int,
        max_input_len: int,
        temperature: float = 1.0,
        c2i=C2I,
        i2c=I2C,
    ):
        self.seed_text = seed_text.lower()
        self.next_chars = next_chars
        self.max_input_len = max_input_len
        self.temperature = temperature
        self.c2i = c2i
        self.i2c = i2c

    def sample(self, preds: np.ndarray):
        preds = tf.math.softmax(preds, axis=-1)
        preds = np.asarray(preds).astype("float64")
        preds = np.log(preds) / self.temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)

    def on_epoch_begin(self, epoch, logs=None):
        generated = ""
        sentence = self.seed_text[-self.max_input_len:]
        print(sentence)
        for _ in range(self.next_chars):
            x_pred = np.zeros((1, self.max_input_len, len(self.c2i)))
            for t, char in enumerate(sentence):
                x_pred[0, t, self.c2i[char]] = 1.0
            preds = self.model.predict_on_batch(x_pred)[0]
            next_index = self.sample(preds)
            next_char = self.i2c[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char
        print("Epoch:", epoch, "Seed:", self.seed_text)
        print("Generated:", generated)
        return generated


In [34]:
model.fit(
    gen,
    steps_per_epoch=len(dataset) // STEP // BATCH_SIZE,
    epochs=10,
    verbose=1,
    callbacks=[
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i", 500, MAX_INPUT_LEN, 0.5, C2I, I2C),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i", 500, MAX_INPUT_LEN, 0.2, C2I, I2C),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i", 500, MAX_INPUT_LEN, 0.1, C2I, I2C),
        tf.keras.callbacks.ModelCheckpoint('../lstm_models/chars/model_best_4.h5', monitor='loss', save_best_only=True, save_weights_only=False)
    ],
)

 temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Epoch: 0 Seed: dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Generated: em  isrpp   odywyaao im a d  t  a a  ed aa,aeu za tns di t      n  oy   a epr en cee e l wg csoe      n     aas  n  a e  dzw     ee y ei    i e  a a  o a    s , ng t o   o ł eo    oz  oioao tsk     w eaysszn rsej a  d odoearw.w oacno iim  nec  eri a    sewyo  eć ł e i  z  w eeta  ac  s e i    yer aae  o w  ea  tły z e  rai.io  n eo   i     t aoaka   n aas   aa  b i m a    e ao w opane   da toeęa    n     ac i a oasaiyina ie c e deca   iasc.d jt  zeo  o o zp adjeist e   y e t cayo w ię  naboa  a 
 temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Epoch: 0 Seed: dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Generated:                             a  z 

In [35]:
pc = PredictCallback("dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. Zabrała se sobą koszyczek z ciastem i", 500, MAX_INPUT_LEN, 0.5, C2I, I2C)
pc.model = model
pc.temperature = 1.2
pc.on_epoch_begin(0)
pc.temperature = 1.0
pc.on_epoch_begin(0)
pc.temperature = 0.5
pc.on_epoch_begin(0)
pc.temperature = 0.2
pc.on_epoch_begin(0)
pc.temperature = 0.1
pc.on_epoch_begin(0)


 temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Epoch: 0 Seed: dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Generated:  wdział w swoim wykonaniu.piotro się sądził w jedneberach, ale cornelchowie, głośno, który z ustcheniu takiego pręzami, siedząco w powhodni, ozwał się trzecia całego ślęczki, sją wciąż dokadywał gamzę!jeżeli w to mdy serce i umoga jednalniejszej: patrzyć nacisków?zapomniała.czekali różne uzdlnienie, że wszyscy się pozewszyli swoją i proroką potwornaki widowisko, jak niektóre wywłóczbały oczy opieki!caremona grzmoty, którą mamy na kwiatów nauczących kecholic genua sam od tego dłużającego rożnymi 
 temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Epoch: 0 Seed: dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. zabrała se sobą koszyczek z ciastem i
Generated:  ludziom powodu dość nie tylko w 

' pod stronę pod podstawę i pod podwórzem po stronie pod przeszkody i pod podróżnym starem pod strony przed nim pod podstawę i pod strony pod podstawym pod podstawym pod strony pod podstawym przez się po prostu i pod podstawym pod strony pod nim po policzku i podobne starego pod podstawym powietrzem w podobne szczęście pod podstawym i przed sobą przed nim w podobne strony i pod strony pod wielkim pod podwórzem i pod nim pod nim w szerokiej stronie pod przeszkody i pod strony pod podstawym pod str'

In [42]:
from keras.models import load_model, Model
model: Model = load_model('../lstm_models/chars/model_best_4.h5')

for layer in model.layers[:1]:
    layer.trainable = False

model.summary()

dataset_fin = load_tokenized_sentences("../datasets/pickled/chars/bajki-extend_str.pickle")
gen_fin = text_generator(
    dataset_fin, C2I, MAX_INPUT_LEN, 2, shuffle=True, batch_size=BATCH_SIZE, sparse=True
)

model.fit(
    gen_fin,
    steps_per_epoch=len(dataset_fin) // STEP // BATCH_SIZE,
    epochs=10,
    verbose=1,
    callbacks=[
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. Zabrała se sobą koszyczek z ciastem i", 500, MAX_INPUT_LEN, 0.5, C2I, I2C),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. Zabrała se sobą koszyczek z ciastem i", 500, MAX_INPUT_LEN, 0.2, C2I, I2C),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu, aby odwiedzić babcię. Zabrała se sobą koszyczek z ciastem i", 500, MAX_INPUT_LEN, 0.1, C2I, I2C),
        tf.keras.callbacks.ModelCheckpoint('../lstm_models/chars/model_best_4_retrain.h5', monitor='loss', save_best_only=True, save_weights_only=False)
    ],
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 100, 256)         177152    
 nal)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 44)                11308     
                                                                 
Total params: 779,564
Trainable params: 602,412
Non-trainable params: 177,152
__________________________________________

In [ ]:
pc = PredictCallback("tego dnia wieczorem, kiedy maciuś zaganiał stado do sołtysowego obejścia, krasula z łaciatą zaczęły", 500, MAX_INPUT_LEN, 0.5, C2I, I2C)
pc.model = model
pc.temperature = 1.2
pc.on_epoch_begin(0)
pc.temperature = 1.0
pc.on_epoch_begin(0)
pc.temperature = 0.5
pc.on_epoch_begin(0)
pc.temperature = 0.2
pc.on_epoch_begin(0)
pc.temperature = 0.1
pc.on_epoch_begin(0)